# Detecção Automática de Objetos em Lâminas

Pipeline 100% automático:
1. SAM detecta TODOS os objetos
2. Filtros morfológicos removem ruído/background
3. Extrai e salva cada objeto separadamente
4. Pronto para clusterização no SAMI

**Zero prompts. Zero intervenção.**

## 1. Setup

In [ ]:
# Instalar dependências
# !pip install segment-anything opencv-python numpy matplotlib torch torchvision

In [ ]:
# Download do modelo SAM (executar uma vez)
# !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O sam_vit_b.pth
# !echo "Download completo: sam_vit_b.pth (375MB)"

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from dataclasses import dataclass
from typing import List, Tuple, Optional
import json
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

## 2. Configuração (ajuste uma vez)

In [ ]:
@dataclass
class Config:
    """Configuração do pipeline - ajuste conforme suas imagens"""
    
    # Paths
    sam_checkpoint: str = "sam_vit_b.pth"
    input_dir: str = "./input_images"      # Onde estão as lâminas
    output_dir: str = "./detected_objects"  # Onde salvar objetos extraídos
    
    # Filtros de área (em pixels)
    min_area: int = 5000        # Objetos menores são descartados
    max_area_ratio: float = 0.6  # Máximo 60% da imagem (remove background)
    
    # Filtros morfológicos
    min_solidity: float = 0.3    # Quão "sólido" é o objeto (0-1)
    min_aspect_ratio: float = 1.2 # Fósseis são geralmente alongados
    max_aspect_ratio: float = 15.0
    
    # SAM parameters
    points_per_side: int = 32    # Densidade de pontos (mais = mais detecções)
    pred_iou_thresh: float = 0.86
    stability_score_thresh: float = 0.92
    
    # Output
    padding: int = 20            # Margem ao redor do objeto extraído
    save_masks: bool = True      # Salvar máscaras junto com ROIs
    save_visualization: bool = True

config = Config()

## 3. Core: Detector Automático

In [ ]:
@dataclass
class DetectedObject:
    """Objeto detectado na lâmina"""
    id: int
    bbox: Tuple[int, int, int, int]  # x1, y1, x2, y2
    mask: np.ndarray
    area: int
    centroid: Tuple[int, int]
    aspect_ratio: float
    solidity: float
    iou_score: float
    stability_score: float
    roi_image: Optional[np.ndarray] = None
    roi_mask: Optional[np.ndarray] = None

In [ ]:
class AutomaticObjectDetector:
    """
    Detector 100% automático usando SAM.
    Detecta todos os objetos e filtra por propriedades morfológicas.
    """
    
    def __init__(self, config: Config):
        self.config = config
        self._load_sam()
    
    def _load_sam(self):
        """Carrega modelo SAM"""
        from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
        
        print("Carregando SAM...")
        
        # Detectar tipo do modelo pelo nome do arquivo
        checkpoint = self.config.sam_checkpoint
        if 'vit_h' in checkpoint:
            model_type = 'vit_h'
        elif 'vit_l' in checkpoint:
            model_type = 'vit_l'
        else:
            model_type = 'vit_b'
        
        sam = sam_model_registry[model_type](checkpoint=checkpoint)
        sam.to(DEVICE)
        
        self.mask_generator = SamAutomaticMaskGenerator(
            sam,
            points_per_side=self.config.points_per_side,
            pred_iou_thresh=self.config.pred_iou_thresh,
            stability_score_thresh=self.config.stability_score_thresh,
            min_mask_region_area=self.config.min_area,
        )
        
        print(f"SAM carregado ({model_type}) em {DEVICE}")
    
    def _compute_morphological_features(self, mask: np.ndarray) -> dict:
        """Calcula features morfológicas de uma máscara"""
        # Encontrar contornos
        contours, _ = cv2.findContours(
            mask.astype(np.uint8), 
            cv2.RETR_EXTERNAL, 
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        if not contours:
            return None
        
        # Pegar maior contorno
        contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(contour)
        
        if area < 100:
            return None
        
        # Bounding box
        x, y, w, h = cv2.boundingRect(contour)
        
        # Aspect ratio
        aspect_ratio = max(w, h) / (min(w, h) + 1e-6)
        
        # Solidity (área / área do convex hull)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = area / (hull_area + 1e-6)
        
        # Centroid
        M = cv2.moments(contour)
        if M["m00"] > 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = x + w // 2, y + h // 2
        
        return {
            'bbox': (x, y, x + w, y + h),
            'area': area,
            'aspect_ratio': aspect_ratio,
            'solidity': solidity,
            'centroid': (cx, cy),
            'contour': contour
        }
    
    def _filter_detection(self, features: dict, img_area: int) -> bool:
        """Verifica se detecção passa nos filtros"""
        if features is None:
            return False
        
        # Filtro de área
        if features['area'] < self.config.min_area:
            return False
        
        if features['area'] > img_area * self.config.max_area_ratio:
            return False
        
        # Filtro de aspect ratio
        if features['aspect_ratio'] < self.config.min_aspect_ratio:
            return False
        
        if features['aspect_ratio'] > self.config.max_aspect_ratio:
            return False
        
        # Filtro de solidity
        if features['solidity'] < self.config.min_solidity:
            return False
        
        return True
    
    def _extract_roi(self, img: np.ndarray, mask: np.ndarray, 
                     bbox: Tuple[int, int, int, int]) -> Tuple[np.ndarray, np.ndarray]:
        """Extrai ROI com padding"""
        x1, y1, x2, y2 = bbox
        h, w = img.shape[:2]
        pad = self.config.padding
        
        # Aplicar padding
        x1 = max(0, x1 - pad)
        y1 = max(0, y1 - pad)
        x2 = min(w, x2 + pad)
        y2 = min(h, y2 + pad)
        
        roi_img = img[y1:y2, x1:x2].copy()
        roi_mask = mask[y1:y2, x1:x2].copy()
        
        return roi_img, roi_mask
    
    def detect(self, img: np.ndarray) -> List[DetectedObject]:
        """
        Detecta todos os objetos na imagem.
        
        Args:
            img: Imagem RGB (numpy array)
            
        Returns:
            Lista de objetos detectados
        """
        h, w = img.shape[:2]
        img_area = h * w
        
        # SAM gera todas as máscaras
        masks_data = self.mask_generator.generate(img)
        
        objects = []
        obj_id = 0
        
        for mask_info in masks_data:
            mask = mask_info['segmentation']
            
            # Calcular features morfológicas
            features = self._compute_morphological_features(mask)
            
            # Aplicar filtros
            if not self._filter_detection(features, img_area):
                continue
            
            # Extrair ROI
            roi_img, roi_mask = self._extract_roi(img, mask, features['bbox'])
            
            obj = DetectedObject(
                id=obj_id,
                bbox=features['bbox'],
                mask=mask,
                area=features['area'],
                centroid=features['centroid'],
                aspect_ratio=features['aspect_ratio'],
                solidity=features['solidity'],
                iou_score=mask_info['predicted_iou'],
                stability_score=mask_info['stability_score'],
                roi_image=roi_img,
                roi_mask=roi_mask
            )
            
            objects.append(obj)
            obj_id += 1
        
        # Ordenar por área (maior primeiro)
        objects.sort(key=lambda x: x.area, reverse=True)
        
        # Reatribuir IDs após ordenação
        for i, obj in enumerate(objects):
            obj.id = i
        
        return objects

## 4. Pipeline de Processamento em Batch

In [ ]:
class BatchProcessor:
    """
    Processa múltiplas imagens automaticamente.
    Salva todos os objetos detectados organizados por imagem.
    """
    
    def __init__(self, config: Config):
        self.config = config
        self.detector = AutomaticObjectDetector(config)
        
        # Criar diretórios de saída
        self.output_path = Path(config.output_dir)
        self.output_path.mkdir(parents=True, exist_ok=True)
        
        (self.output_path / 'rois').mkdir(exist_ok=True)
        if config.save_masks:
            (self.output_path / 'masks').mkdir(exist_ok=True)
        if config.save_visualization:
            (self.output_path / 'visualizations').mkdir(exist_ok=True)
    
    def _load_image(self, path: str) -> np.ndarray:
        """Carrega imagem em RGB"""
        img = cv2.imread(str(path))
        if img is None:
            raise ValueError(f"Não foi possível carregar: {path}")
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    def _save_object(self, obj: DetectedObject, image_name: str):
        """Salva um objeto detectado"""
        prefix = f"{image_name}_obj{obj.id:03d}"
        
        # Salvar ROI
        roi_path = self.output_path / 'rois' / f"{prefix}.png"
        cv2.imwrite(str(roi_path), cv2.cvtColor(obj.roi_image, cv2.COLOR_RGB2BGR))
        
        # Salvar máscara
        if self.config.save_masks:
            mask_path = self.output_path / 'masks' / f"{prefix}_mask.png"
            cv2.imwrite(str(mask_path), (obj.roi_mask * 255).astype(np.uint8))
    
    def _save_visualization(self, img: np.ndarray, objects: List[DetectedObject], 
                            image_name: str):
        """Salva visualização com todas as detecções"""
        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
        
        # Imagem com bboxes
        vis = img.copy()
        for obj in objects:
            x1, y1, x2, y2 = obj.bbox
            cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(vis, f"#{obj.id}", (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
        
        axes[0].imshow(vis)
        axes[0].set_title(f"{image_name}: {len(objects)} objetos detectados")
        axes[0].axis('off')
        
        # Máscaras combinadas
        overlay = img.copy().astype(float)
        colors = plt.cm.rainbow(np.linspace(0, 1, max(len(objects), 1)))
        
        for obj, color in zip(objects, colors):
            mask_3d = np.stack([obj.mask] * 3, axis=-1)
            overlay = np.where(mask_3d, 
                              overlay * 0.5 + np.array(color[:3]) * 255 * 0.5, 
                              overlay)
        
        axes[1].imshow(overlay.astype(np.uint8))
        axes[1].set_title("Máscaras de segmentação")
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.savefig(self.output_path / 'visualizations' / f"{image_name}_detection.png",
                   dpi=150, bbox_inches='tight')
        plt.close()
    
    def _save_metadata(self, all_results: dict):
        """Salva metadados de todas as detecções"""
        metadata = {
            'config': {
                'min_area': self.config.min_area,
                'max_area_ratio': self.config.max_area_ratio,
                'min_solidity': self.config.min_solidity,
                'min_aspect_ratio': self.config.min_aspect_ratio,
                'max_aspect_ratio': self.config.max_aspect_ratio,
            },
            'images': {}
        }
        
        for image_name, objects in all_results.items():
            metadata['images'][image_name] = {
                'n_objects': len(objects),
                'objects': [
                    {
                        'id': obj.id,
                        'bbox': obj.bbox,
                        'area': obj.area,
                        'aspect_ratio': round(obj.aspect_ratio, 2),
                        'solidity': round(obj.solidity, 3),
                        'iou_score': round(obj.iou_score, 3),
                        'roi_file': f"{image_name}_obj{obj.id:03d}.png"
                    }
                    for obj in objects
                ]
            }
        
        with open(self.output_path / 'metadata.json', 'w') as f:
            json.dump(metadata, f, indent=2)
    
    def process(self, image_paths: List[str] = None) -> dict:
        """
        Processa todas as imagens.
        
        Args:
            image_paths: Lista de caminhos (ou None para usar input_dir)
            
        Returns:
            Dict com resultados por imagem
        """
        # Listar imagens
        if image_paths is None:
            input_path = Path(self.config.input_dir)
            image_paths = list(input_path.glob('*.jpg')) + \
                         list(input_path.glob('*.JPG')) + \
                         list(input_path.glob('*.jpeg')) + \
                         list(input_path.glob('*.png'))
        
        print(f"\n{'='*60}")
        print(f"PROCESSAMENTO AUTOMÁTICO")
        print(f"{'='*60}")
        print(f"Imagens encontradas: {len(image_paths)}")
        print(f"Output: {self.output_path}")
        print(f"{'='*60}\n")
        
        all_results = {}
        total_objects = 0
        
        for img_path in tqdm(image_paths, desc="Processando"):
            image_name = Path(img_path).stem
            
            try:
                # Carregar imagem
                img = self._load_image(img_path)
                
                # Detectar objetos
                objects = self.detector.detect(img)
                
                # Salvar cada objeto
                for obj in objects:
                    self._save_object(obj, image_name)
                
                # Salvar visualização
                if self.config.save_visualization:
                    self._save_visualization(img, objects, image_name)
                
                all_results[image_name] = objects
                total_objects += len(objects)
                
                print(f"  {image_name}: {len(objects)} objetos")
                
            except Exception as e:
                print(f"  ERRO em {image_name}: {e}")
                all_results[image_name] = []
        
        # Salvar metadados
        self._save_metadata(all_results)
        
        # Resumo
        print(f"\n{'='*60}")
        print(f"RESUMO")
        print(f"{'='*60}")
        print(f"Imagens processadas: {len(all_results)}")
        print(f"Total de objetos: {total_objects}")
        print(f"Média por imagem: {total_objects/max(len(all_results),1):.1f}")
        print(f"\nArquivos salvos em: {self.output_path}")
        print(f"  - rois/: {total_objects} imagens de objetos")
        if self.config.save_masks:
            print(f"  - masks/: {total_objects} máscaras")
        if self.config.save_visualization:
            print(f"  - visualizations/: {len(all_results)} visualizações")
        print(f"  - metadata.json: metadados completos")
        print(f"{'='*60}")
        
        return all_results

## 5. Preparação para SAMI (Clusterização)

In [ ]:
def prepare_for_sami(detected_objects_dir: str, 
                     sami_dataset_dir: str,
                     class_name: str = "unknown"):
    """
    Organiza objetos detectados no formato ImageFolder para SAMI.
    
    Estrutura de saída:
    sami_dataset_dir/
    └── unknown/
        ├── image1_obj000.png
        ├── image1_obj001.png
        └── ...
    
    Depois de rodar clusterização no SAMI, você pode reorganizar
    por cluster para treinar classificadores.
    """
    import shutil
    
    src = Path(detected_objects_dir) / 'rois'
    dst = Path(sami_dataset_dir) / class_name
    dst.mkdir(parents=True, exist_ok=True)
    
    # Copiar todas as ROIs
    count = 0
    for img_file in src.glob('*.png'):
        shutil.copy(img_file, dst / img_file.name)
        count += 1
    
    print(f"Copiados {count} objetos para {dst}")
    print(f"\nPronto para rodar SAMI:")
    print(f"  from SAMI import run_full_evaluation")
    print(f"  results = run_full_evaluation('{sami_dataset_dir}')")
    
    return dst

---
## 6. EXECUTAR

In [ ]:
# ============================================
# CONFIGURAÇÃO - AJUSTE AQUI
# ============================================

config = Config(
    # Paths
    sam_checkpoint="sam_vit_b.pth",     # Caminho para o modelo SAM
    input_dir="./input_images",          # Pasta com suas lâminas
    output_dir="./detected_objects",     # Onde salvar resultados
    
    # Filtros - ajuste conforme suas imagens
    min_area=5000,           # Área mínima em pixels
    max_area_ratio=0.6,      # Máximo 60% da imagem
    min_solidity=0.3,        # Objetos "sólidos"
    min_aspect_ratio=1.2,    # Objetos alongados
    max_aspect_ratio=15.0,
    
    # SAM
    points_per_side=32,      # Mais pontos = mais detecções (mais lento)
    
    # Output
    padding=20,
    save_masks=True,
    save_visualization=True
)

In [ ]:
# ============================================
# PROCESSAR TODAS AS IMAGENS
# ============================================

processor = BatchProcessor(config)
results = processor.process()

In [ ]:
# ============================================
# PREPARAR PARA SAMI
# ============================================

prepare_for_sami(
    detected_objects_dir="./detected_objects",
    sami_dataset_dir="./sami_dataset",
    class_name="detected_objects"
)

---
## 7. Visualizar Resultados

In [ ]:
def show_detected_objects(output_dir: str, max_objects: int = 20):
    """Mostra grid com objetos detectados"""
    rois_dir = Path(output_dir) / 'rois'
    roi_files = list(rois_dir.glob('*.png'))[:max_objects]
    
    if not roi_files:
        print("Nenhum objeto encontrado")
        return
    
    n_cols = 5
    n_rows = (len(roi_files) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for i, ax in enumerate(axes):
        if i < len(roi_files):
            img = cv2.imread(str(roi_files[i]))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ax.imshow(img)
            ax.set_title(roi_files[i].stem, fontsize=8)
        ax.axis('off')
    
    plt.suptitle(f"Objetos Detectados ({len(roi_files)} de {len(list(rois_dir.glob('*.png')))})", 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# show_detected_objects("./detected_objects")

In [ ]:
def analyze_detections(output_dir: str):
    """Análise estatística das detecções"""
    with open(Path(output_dir) / 'metadata.json') as f:
        metadata = json.load(f)
    
    all_objects = []
    for image_name, data in metadata['images'].items():
        for obj in data['objects']:
            obj['image'] = image_name
            all_objects.append(obj)
    
    if not all_objects:
        print("Nenhum objeto detectado")
        return
    
    # Estatísticas
    areas = [o['area'] for o in all_objects]
    aspects = [o['aspect_ratio'] for o in all_objects]
    solidities = [o['solidity'] for o in all_objects]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].hist(areas, bins=30, edgecolor='black')
    axes[0].set_xlabel('Área (pixels)')
    axes[0].set_ylabel('Frequência')
    axes[0].set_title('Distribuição de Área')
    
    axes[1].hist(aspects, bins=30, edgecolor='black')
    axes[1].set_xlabel('Aspect Ratio')
    axes[1].set_title('Distribuição de Aspect Ratio')
    
    axes[2].hist(solidities, bins=30, edgecolor='black')
    axes[2].set_xlabel('Solidity')
    axes[2].set_title('Distribuição de Solidity')
    
    plt.suptitle(f"Estatísticas de {len(all_objects)} objetos detectados", 
                fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"\nEstatísticas:")
    print(f"  Área: {np.mean(areas):.0f} ± {np.std(areas):.0f} pixels")
    print(f"  Aspect Ratio: {np.mean(aspects):.2f} ± {np.std(aspects):.2f}")
    print(f"  Solidity: {np.mean(solidities):.3f} ± {np.std(solidities):.3f}")

# analyze_detections("./detected_objects")

---
## Notas

### Workflow Completo:

```
1. Colocar lâminas em ./input_images/
2. Executar este notebook (células 6)
3. Verificar ./detected_objects/visualizations/
4. Se necessário, ajustar filtros e re-executar
5. Rodar SAMI para clusterização
```

### Parâmetros Importantes:

| Parâmetro | Efeito | Ajuste se... |
|-----------|--------|-------------|
| `min_area` | Filtra objetos pequenos | Detectando muito ruído |
| `max_area_ratio` | Filtra objetos grandes | Detectando background |
| `min_aspect_ratio` | Exige objetos alongados | Fósseis são alongados |
| `min_solidity` | Exige formas compactas | Filtra artefatos fragmentados |
| `points_per_side` | Densidade de detecção | Mais = mais detecções (mais lento) |

### Se estiver detectando muito background:
- Aumentar `min_area`
- Diminuir `max_area_ratio`
- Aumentar `min_solidity`

### Se estiver perdendo fósseis:
- Diminuir `min_area`
- Diminuir `min_aspect_ratio`
- Aumentar `points_per_side`